<a href="https://cognitiveclass.ai"><img src = "https://ibm.box.com/shared/static/9gegpsmnsoo25ikkbl4qzlvlyjbgxs5x.png" width = 400> </a>

<h1 align=center><font size = 5>Learning FourSquare API with Python</font></h1>

## Introduction

In this lab, you will learn in details how to make calls to the Foursquare API for different purposes. You will learn how to construct a URL to send a request to the API to search for a specific type of venues, to explore a particular venue, to explore a Foursquare user, to explore a geographical location, and to get trending venues around a location. Also, you will learn how to use the visualization library, Folium, to visualize the results.

## Table of Contents

1. <a href="#item1">Foursquare API Search Function</a>
2. <a href="#item2">Explore a Given Venue</a>  
3. <a href="#item3">Explore a User</a>  
4. <a href="#item4">Foursquare API Explore Function</a>  
5. <a href="#item5">Get Trending Venues</a>  

### Import necessary Libraries

In [6]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# !conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

# !conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Folium installed
Libraries imported.


### Define Foursquare Credentials and Version

##### Make sure that you have created a Foursquare developer account and have your credentials handy

In [7]:
CLIENT_ID = 'S3HAVINQP123TLZ0TXXDM1TOS0RV245HAUVXDUPYMMKWEJZR' # your Foursquare ID
CLIENT_SECRET = 'OQ1YSTUNXYFED0TCKILUQB0UWDOLO24QAEHDPMKLDYN3YCQZ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: S3HAVINQP123TLZ0TXXDM1TOS0RV245HAUVXDUPYMMKWEJZR
CLIENT_SECRET:OQ1YSTUNXYFED0TCKILUQB0UWDOLO24QAEHDPMKLDYN3YCQZ


#### Let's again assume that you are staying at the Conrad hotel. So let's start by converting the Contrad Hotel's address to its latitude and longitude coordinates.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>foursquare_agent</em>, as shown below.

In [8]:
address = '102 North End Ave, New York, NY'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7151482 -74.0156573


<a id="item1"></a>

## 1. Search for a specific venue category
> `https://api.foursquare.com/v2/venues/`**search**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&query=`**QUERY**`&radius=`**RADIUS**`&limit=`**LIMIT**

#### Now, let's assume that it is lunch time, and you are craving Italian food. So, let's define a query to search for Italian food that is within 500 metres from the Conrad Hotel. 

In [9]:
search_query = 'Italian'
radius = 500
print(search_query + ' .... OK!')

Italian .... OK!


#### Define the corresponding URL

In [14]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=S3HAVINQP123TLZ0TXXDM1TOS0RV245HAUVXDUPYMMKWEJZR&client_secret=OQ1YSTUNXYFED0TCKILUQB0UWDOLO24QAEHDPMKLDYN3YCQZ&ll=40.7151482,-74.0156573&v=20180604&query=Italian&radius=500&limit=30'

#### Send the GET Request and examine the results

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eca89ee0cc1fd001b96e865'},
 'response': {'venues': [{'id': '4fa862b3e4b0ebff2f749f06',
    'name': "Harry's Italian Pizza Bar",
    'location': {'address': '225 Murray St',
     'lat': 40.71521779064671,
     'lng': -74.01473940209351,
     'labeledLatLngs': [{'label': 'display',
       'lat': 40.71521779064671,
       'lng': -74.01473940209351},
      {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
     'distance': 77,
     'postalCode': '10282',
     'cc': 'US',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['225 Murray St',
      'New York, NY 10282',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1ca941735',
      'name': 'Pizza Place',
      'pluralName': 'Pizza Places',
      'shortName': 'Pizza',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-15903320

#### Get relevant part of JSON and transform it into a *pandas* dataframe

In [17]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
dataframe.head()

,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4fa862b3e4b0ebff2f749f06,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",v-1590332018,False,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",77,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta..."
1,4f3232e219836c91c7bfde94,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",v-1590332018,False,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",499,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta..."


In [18]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered

,name,categories,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,id
0,Harry's Italian Pizza Bar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",77,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,"[{'id': '4d4b7105d754a06374d81259', 'name': 'F...",63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",499,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


#### Define information of interest and filter dataframe

In [19]:

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Harry's Italian Pizza Bar,Pizza Place,225 Murray St,40.715218,-74.014739,"[{'label': 'display', 'lat': 40.71521779064671...",77,10282,US,New York,NY,United States,"[225 Murray St, New York, NY 10282, United Sta...",4fa862b3e4b0ebff2f749f06
1,Conca Cucina Italian Restaurant,Food,63 W Broadway,40.714484,-74.009806,"[{'label': 'display', 'lat': 40.71448400000000...",499,10007,US,New York,NY,United States,"[63 W Broadway, New York, NY 10007, United Sta...",4f3232e219836c91c7bfde94


#### Let's visualize the Italian restaurants that are nearby

In [ ]:
dataframe_filtered.name

In [20]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

<a id="item2"></a>

## 2. Explore a Given Venue
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### A. Let's explore the closest Italian restaurant -- _Harry's Italian Pizza Bar_

In [21]:
venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4fa862b3e4b0ebff2f749f06?client_id=S3HAVINQP123TLZ0TXXDM1TOS0RV245HAUVXDUPYMMKWEJZR&client_secret=OQ1YSTUNXYFED0TCKILUQB0UWDOLO24QAEHDPMKLDYN3YCQZ&v=20180604'

#### Send GET request for result

In [22]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'price', 'hasMenu', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'menu', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4fa862b3e4b0ebff2f749f06',
 'name': "Harry's Italian Pizza Bar",
 'contact': {'phone': '2126081007', 'formattedPhone': '(212) 608-1007'},
 'location': {'address': '225 Murray St',
  'lat': 40.71521779064671,
  'lng': -74.01473940209351,
  'labeledLatLngs': [{'label': 'display',
    'lat': 40.71521779064671,
    'lng': -74.01473940209351},
   {'label': 'entrance', 'lat': 40.715361, 'lng': -74.014975}],
  'postalCode': '10282',
  'cc': 'US',
  'city': 'New York',
  'state': 'NY',
  'country': 'United States',
  'formattedAddress': ['225 Murray St',
   'New York, NY 10282',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/harrys-italian-pizza-bar/4fa862b3e4b0ebff2f749f06',
 'categories': [{'id': '4bf58dd8d48988d1ca941735',
   'name': 'Pizza Place',
   'pluralName': 'Pizza Places',
   'shortName': 'Pizza',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/pizza_',
    'suffix': '.png'},
   'primary': True},
  {'id': '4bf58dd8d48988d110941735',
   'n

### B. Get the venue's overall rating

In [23]:
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

6.4


That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

In [24]:
venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


Since this restaurant has no ratings, let's check the third restaurant.

In [25]:
venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

7.4


Since this restaurant has a slightly better rating, let's explore it further.

### C. Get the number of tips

In [26]:
result['response']['venue']['tips']['count']

19

### D. Get the venue's tips
> `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

#### Create URL and send GET request. Make sure to set limit to get all tips

In [27]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eca8c1f882fc7001b533638'},
 'response': {'tips': {'count': 19,
   'items': [{'id': '5ab1cb46c9a517174651d3fe',
     'createdAt': 1521601350,
     'text': 'A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites',
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/5ab1cb46c9a517174651d3fe',
     'lang': 'en',
     'likes': {'count': 0, 'groups': []},
     'logView': True,
     'agreeCount': 4,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '484542633',
      'firstName': 'Nick',
      'lastName': 'E',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'}},
     'authorInteractionType': 'liked'},
    {'id': '5cb7051b180b910039f90625',
     'createdAt': 1555498267,
     

#### Get tips and list of associated features

In [28]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'lang', 'likes', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user', 'authorInteractionType'])

#### Format column width and display all tips

In [32]:
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,"A+ Italian food! Trust me on this: my mom’s side of the family is 100% Italian. I was born and bred to know good pasta when I see it, and Ecco is one of my all-time NYC favorites",4,0,5ab1cb46c9a517174651d3fe,Nick,E,484542633
1,"Excellent food!! Osso bucco special one of the best I ever had! Lobster ravioli with porcini mushroomhomemade Italian cheesecake, tiramisu and napoleons...calamari fra diavolo was sautéed not fried",1,0,5cb7051b180b910039f90625,Lynn,B,446298346


Now remember that because we are using a personal developer account, then we can access only 2 of the restaurant's tips, instead of all 15 tips.

<a id="item3"></a>

## 3. Search a Foursquare User
> `https://api.foursquare.com/v2/users/`**USER_ID**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**

### Define URL, send GET request and display features associated with user

In [37]:

user_id = '484542633' # user ID with most agree counts and complete profile
ACCESS_TOKEN = 'BJ3NCUYI4MA50OTTCYCLK24MOFE2PRSBODXVIKNIVMU2XCP4'

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN, VERSION) # define URL

# send GET request
results = requests.get(url).json()
user_data = results['response']['user']

# display features associated with user
user_data.keys()

dict_keys(['id', 'firstName', 'lastName', 'gender', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'checkinPings', 'pings', 'type', 'mayorships', 'checkins', 'lists', 'blockedStatus', 'lenses'])

In [38]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

First Name: Nick
Last Name: El-Tawil
Home City: New York, NY


#### How many tips has this user submitted?

In [39]:
user_data['tips']

{'count': 98}

Wow! So it turns out that Nick is a very active Foursquare user, with more than 250 tips.

### Get User's tips

In [41]:
# define tips URL
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

# send GET request and get user's tips
results = requests.get(url).json()
tips = results['response']['tips']['items']

# format column width
pd.set_option('display.max_colwidth', None)

tips_df = pd.json_normalize(tips)

# filter columns
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display user's tips
tips_filtered

,text,agreeCount,disagreeCount,id
0,They serve coffee!!!!!!,1,0,5accc98c0313204c9d7ec157
1,"Quick, cheap lunch that tastes good! Way shorter line than Chipotle, too.",2,0,5acbec70a0215b732e264fe8
2,You’re not a real New Yorker until you’ve shame-ordered Insomnia Cookies for delivery at 3am,1,0,5acbbd4eb1538e45373b07f5
3,Good for you yet still tasty! Clean green protein is my go-to after I hit the gym 💪,2,0,5acbbcda01235808d5d6dc75
4,Burger game strong 💪,1,0,5ab575fb6bdee65f759da8c1
5,"Great burgers & fries! Also, this place is exactly what it’s like when you go to a bar in the Southwest. Source: I’m from Arizona.",2,0,5ab5575d73fe2516ad8f363b
6,Açaí bowl + peanut butter + whey protein = 💪💪💪,1,0,5ab42db53c858d64af2688a4
7,Highly underrated and way less crowded than Central Park!,3,0,5ab42c396f706a29f53ad1a8
8,"Way easier to navigate than the Met proper, plus the Met Breuer focuses on modern art. If I only have a limited amount of time to spend in a museum, I would rather go here than anywhere else!",6,0,5ab42b987dc9e17930e5ff5b
9,Get the açaí bowl with peanut butter after your work out and thank me later 👌,1,0,5ab42aca2a7ab6333652b266


#### Let's get the venue for the tip with the greatest number of agree counts

In [42]:
tip_id = '5ab5575d73fe2516ad8f363b' # tip id

# define URL
url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

# send GET Request and examine results
result = requests.get(url).json()
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])

Cowgirl
{'address': '519 Hudson St', 'crossStreet': 'at W 10th St', 'lat': 40.73373338282062, 'lng': -74.0062998849649, 'labeledLatLngs': [{'label': 'display', 'lat': 40.73373338282062, 'lng': -74.0062998849649}], 'postalCode': '10014', 'cc': 'US', 'city': 'New York', 'state': 'NY', 'country': 'United States', 'formattedAddress': ['519 Hudson St (at W 10th St)', 'New York, NY 10014', 'United States']}


### Get User's friends

In [44]:
user_friends = pd.json_normalize(user_data['friends']['groups'][0]['items'])
user_friends

""


Interesting. Despite being very active, it turns out that Nick does not have any friends on Foursquare. This might definitely change in the future.

### Retrieve the User's Profile Image

In [45]:
user_data

{'id': '484542633',
 'firstName': 'Nick',
 'lastName': 'El-Tawil',
 'gender': 'male',
 'canonicalUrl': 'https://foursquare.com/user/484542633',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/484542633_unymNUmw_FdPs3GjXHujmHcYnN4hf8kEPADlOZuIrdcdm97VX3tFqL7fFNMNA_8Gl9NlU1GYg.jpg'},
 'friends': {'count': 0,
  'groups': [{'type': 'friends',
    'name': 'Mutual friends',
    'count': 0,
    'items': []},
   {'type': 'others', 'name': 'Other friends', 'count': 0, 'items': []}]},
 'tips': {'count': 98},
 'homeCity': 'New York, NY',
 'bio': 'https://www.tawil.team/nick-el-tawil/',
 'contact': {'twitter': 'nickeltawil'},
 'photos': {'count': 2,
  'items': [{'id': '5e41a7ce346b7c00085b0f70',
    'createdAt': 1581361102,
    'source': {'name': 'Foursquare for iOS',
     'url': 'https://foursquare.com/download/#/iphone'},
    'prefix': 'https://fastly.4sqi.net/img/general/',
    'suffix': '/484542633_cF12vc8JQJTd-hyznst-CsHHTdSoi4XwCGKhTuQXNfo.jpg',
    'width': 1440,
   

In [46]:
# 1. grab prefix of photo
# 2. grab suffix of photo
# 3. concatenate them using the image size  
Image(url='https://igx.4sqi.net/img/user/300x300/484542633_mK2Yum7T_7Tn9fWpndidJsmw2Hof_6T5vJBKCHPLMK5OL-U5ZiJGj51iwBstcpDLYa3Zvhvis.jpg')

<a id="item4"></a>

## 4. Explore a location
> `https://api.foursquare.com/v2/venues/`**explore**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**`&limit=`**LIMIT**

#### So, you just finished your gourmet dish at Ecco, and are just curious about the popular spots around the restaurant. In order to explore the area, let's start by getting the latitude and longitude values of Ecco Restaurant.

In [47]:
latitude = 40.715337
longitude = -74.008848

#### Define URL

In [48]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=S3HAVINQP123TLZ0TXXDM1TOS0RV245HAUVXDUPYMMKWEJZR&client_secret=OQ1YSTUNXYFED0TCKILUQB0UWDOLO24QAEHDPMKLDYN3YCQZ&ll=40.715337,-74.008848&v=20180604&radius=500&limit=30'

#### Send GET request and examine results

In [49]:
import requests

In [51]:
results = requests.get(url).json()
print(results)
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

{'meta': {'code': 200, 'requestId': '5eca9495e826ac001ba8619e'}, 'response': {'suggestedFilters': {'header': 'Tap to show:', 'filters': [{'name': 'Open now', 'key': 'openNow'}, {'name': '$-$$$$', 'key': 'price'}]}, 'headerLocation': 'Tribeca', 'headerFullLocation': 'Tribeca, New York', 'headerLocationGranularity': 'neighborhood', 'totalResults': 97, 'suggestedBounds': {'ne': {'lat': 40.7198370045, 'lng': -74.00292208843348}, 'sw': {'lat': 40.710836995499996, 'lng': -74.01477391156652}}, 'groups': [{'type': 'Recommended Places', 'name': 'recommended', 'items': [{'reasons': {'count': 0, 'items': [{'summary': 'This spot is popular', 'type': 'general', 'reasonName': 'globalInteractionReason'}]}, 'venue': {'id': '54148bc6498ea7bb8c05b70a', 'name': 'Juice Press', 'location': {'address': '83 Murray St', 'crossStreet': 'btwn Greenwich St & W Broadway', 'lat': 40.71478769908051, 'lng': -74.0111317502157, 'labeledLatLngs': [{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}

'There are 30 around Ecco restaurant.'

#### Get relevant part of JSON

In [52]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '54148bc6498ea7bb8c05b70a',
  'name': 'Juice Press',
  'location': {'address': '83 Murray St',
   'crossStreet': 'btwn Greenwich St & W Broadway',
   'lat': 40.71478769908051,
   'lng': -74.0111317502157,
   'labeledLatLngs': [{'label': 'display',
     'lat': 40.71478769908051,
     'lng': -74.0111317502157}],
   'distance': 202,
   'postalCode': '10007',
   'cc': 'US',
   'city': 'New York',
   'state': 'NY',
   'country': 'United States',
   'formattedAddress': ['83 Murray St (btwn Greenwich St & W Broadway)',
    'New York, NY 10007',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1d3941735',
    'name': 'Vegetarian / Vegan Restaurant',
    'pluralName': 'Vegetarian / Vegan Restaurants',
    'shortName': 'Vegetarian / Vegan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
  

#### Process JSON and convert it to a clean dataframe

In [53]:
dataframe = pd.json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Juice Press,Vegetarian / Vegan Restaurant,83 Murray St,btwn Greenwich St & W Broadway,40.714788,-74.011132,"[{'label': 'display', 'lat': 40.71478769908051, 'lng': -74.0111317502157}]",202,10007,US,New York,NY,United States,"[83 Murray St (btwn Greenwich St & W Broadway), New York, NY 10007, United States]",NaN,54148bc6498ea7bb8c05b70a
1,Korin,Furniture / Home Store,57 Warren St,Church St,40.714824,-74.009404,"[{'label': 'display', 'lat': 40.71482437714839, 'lng': -74.00940425461492}, {'label': 'entrance', 'lat': 40.714727, 'lng': -74.009399}]",73,10007,US,New York,NY,United States,"[57 Warren St (Church St), New York, NY 10007, United States]",Tribeca,4af5d65ff964a52091fd21e3
2,Los Tacos No. 1,Taco Place,136 Church St,NaN,40.714267,-74.008756,"[{'label': 'display', 'lat': 40.714267, 'lng': -74.008756}]",119,10007,US,New York,NY,United States,"[136 Church St, New York, NY 10007, United States]",NaN,5d5f24ec09484500079aee00
3,Takahachi Bakery,Bakery,25 Murray St,at Church St,40.713653,-74.008804,"[{'label': 'display', 'lat': 40.713652845301894, 'lng': -74.0088038953017}, {'label': 'entrance', 'lat': 40.713716, 'lng': -74.008443}]",187,10007,US,New York,NY,United States,"[25 Murray St (at Church St), New York, NY 10007, United States]",NaN,4c154c9a77cea593c401d260
4,Takahachi,Sushi Restaurant,145 Duane St,btwn W Broadway & Church St,40.716526,-74.008101,"[{'label': 'display', 'lat': 40.71652647412374, 'lng': -74.00810108466207}, {'label': 'entrance', 'lat': 40.716508, 'lng': -74.007989}]",146,10013,US,New York,NY,United States,"[145 Duane St (btwn W Broadway & Church St), New York, NY 10013, United States]",NaN,4a8f2f39f964a520471420e3
5,Equinox Tribeca,Gym,54 Murray St,at W Broadway,40.714099,-74.009686,"[{'label': 'display', 'lat': 40.71409860726041, 'lng': -74.0096857179283}]",154,10007,US,New York,NY,United States,"[54 Murray St (at W Broadway), New York, NY 10007, United States]",NaN,4a6e331af964a52031d41fe3
6,Whole Foods Market,Grocery Store,270 Greenwich Street,at Warren St,40.715579,-74.011368,"[{'label': 'display', 'lat': 40.715579155420606, 'lng': -74.01136823958119}]",214,10007,US,New York,NY,United States,"[270 Greenwich Street (at Warren St), New York, NY 10007, United States]",Tribeca,49bc3b0af964a52020541fe3
7,Mulberry & Vine,Café,73 Warren St,B/t West Broadway & Greenwich,40.715177,-74.010227,"[{'label': 'display', 'lat': 40.71517693966315, 'lng': -74.01022747778285}, {'label': 'entrance', 'lat': 40.715064, 'lng': -74.010146}]",117,10007,US,New York,NY,United States,"[73 Warren St (B/t West Broadway & Greenwich), New York, NY 10007, United States]",NaN,5171b5cc011cef9833bbb787
8,Philip Williams Posters,Antique Shop,122 Chambers St,NaN,40.715284,-74.008781,"[{'label': 'display', 'lat': 40.71528423132827, 'lng': -74.00878093952018}, {'label': 'entrance', 'lat': 40.715188, 'lng': -74.008747}]",8,10007,US,New York,NY,United States,"[122 Chambers St, New York, NY 10007, United States]",NaN,4b747291f964a52042dd2de3
9,The Odeon,French Restaurant,145 W Broadway,at Thomas St,40.717010,-74.008042,"[{'label': 'display', 'lat': 40.71701011409906, 'lng': -74.00804244562225}]",198,10013,US,New York,NY,United States,"[145 W Broadway (at Thomas St), New York, NY 10013, United States]",NaN,3fd66200f964a52083e61ee3


#### Let's visualize these items on the map around our location

In [55]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map

<a id="item5"></a>

## 5. Explore Trending Venues
> `https://api.foursquare.com/v2/venues/`**trending**`?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&ll=`**LATITUDE**`,`**LONGITUDE**`&v=`**VERSION**

#### Now, instead of simply exploring the area around Ecco, you are interested in knowing the venues that are trending at the time you are done with your lunch, meaning the places with the highest foot traffic. So let's do that and get the trending venues around Ecco.

In [54]:
# define URL
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5eca94d0e826ac001ba8f9a8'},
 'response': {'venues': []}}

### Check if any venues are trending at this time

In [56]:
if len(results['response']['venues']) == 0:
    trending_venues_df = 'No trending venues are available at the moment!'
    
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    # filter columns
    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    # filter the category for each row
    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [57]:
# display trending venues
trending_venues_df

'No trending venues are available at the moment!'

Now, depending on when you run the above code, you might get different venues since the venues with the highest foot traffic are fetched live. 

### Visualize trending venues

In [58]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco


    # add Ecco as a red circle mark
    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)


    # add the trending venues as blue circle markers
    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)

In [59]:
# display map
venues_map

'Cannot generate visual as no trending venues are available at the moment!'

<a id="item6"></a>

### Thank you for completing this lab!

This notebook was created by [Alex Aklson](https://www.linkedin.com/in/aklson/). I hope you found this lab interesting and educational. Feel free to contact me if you have any questions!

This notebook is part of a course on **Coursera** called *Applied Data Science Capstone*. If you accessed this notebook outside the course, you can take this course online by clicking [here](http://cocl.us/DP0701EN_Coursera_Week2_LAB1).

<hr>
Copyright &copy; 2018 [Cognitive Class](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).